In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from datetime import timedelta

# خواندن فایل ورودی
df = pd.read_csv('merged_output2.csv')

# تبدیل DATE_MILADI و HOUR به datetime
df['DATE_MILADI'] = pd.to_datetime(df['DATE_MILADI'])
df['datetime'] = df['DATE_MILADI'] + pd.to_timedelta(df['HOUR'], unit='h')

# مرتب‌سازی بر اساس datetime
df = df.sort_values('datetime').reset_index(drop=True)

# چک کردن duplicate در datetime
if df['datetime'].duplicated().any():
    print("هشدار: datetimeهای تکراری وجود دارد. در حال aggregation با میانگین POWER...")
    df = df.groupby('datetime').agg({
        'POWER': 'mean',
        'DATE_MILADI': 'first',
        'HOUR': 'first',
        'DATE_SHAMSI': 'first',
        'CODE': 'first',
        'UNIT_NO': 'first',
        'DAMA': 'mean',
        'ROTOOBAT': 'mean',
        '12209_G13': 'mean',
        '12210_G13': 'mean',
        'ebraz': 'first',
        'importance_factor': 'first'
    }).reset_index()

# استخراج year, month, dayofweek
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek

# فیلتر داده آموزشی (2022 تا 2023) و تست (از 2024 به بعد)
train_mask = df['year'].between(2022, 2023)
test_mask = df['year'] >= 2024

train = df[train_mask]
test = df[test_mask].copy()

# ایجاد سری زمانی POWER با index datetime
power_series_full = df.set_index('datetime')['POWER']

# تنظیم فرکانس ساعتی با forward fill
power_series_full = power_series_full.asfreq('h', method='ffill')

# اضافه کردن ستون DECLARED به تست
test['DECLARED'] = 0.0

# لوپ روی ردیف‌های تست برای پیش‌بینی سه روز آینده با Holt-Winters
for idx, row in test.iterrows():
    current_datetime = row['datetime']
    
    if row['ebraz'] == 0:
        test.at[idx, 'DECLARED'] = 0
        continue
    
    # زمان پایه برای آموزش: دقیقاً سه روز قبل
    train_end_time = current_datetime - timedelta(days=3)
    
    # داده‌های موجود تا train_end_time
    train_data = power_series_full[power_series_full.index <= train_end_time]
    
    if len(train_data) < 168:  # حداقل یک هفته برای فصل روزانه
        test.at[idx, 'DECLARED'] = 0
        continue
    
    # چک کردن وجود مقدار صفر یا منفی در train_data
    if (train_data <= 0).any():
        # اگر صفر یا منفی وجود دارد، از مدل Additive استفاده می‌کنیم
        print(f"مقدار صفر/منفی در داده آموزشی برای {current_datetime} — استفاده از Additive")
        model = ExponentialSmoothing(
            train_data,
            trend='add',
            seasonal='add',          # تغییر به additive برای جلوگیری از خطا
            seasonal_periods=24
        ).fit()
    else:
        # اگر همه مثبت هستند، از Multiplicative استفاده می‌کنیم (بهتر برای تولید برق)
        model = ExponentialSmoothing(
            train_data,
            trend='add',
            seasonal='mul',
            seasonal_periods=24
        ).fit()
    
    # پیش‌بینی دقیقاً 72 گام جلو (3 روز)
    forecast = model.forecast(steps=72)
    
    # مقدار پیش‌بینی برای ساعت مربوطه
    predicted_value = forecast.iloc[-1]
    
    # اگر پیش‌بینی منفی شد (ممکن در additive)، آن را به صفر محدود می‌کنیم
    test.at[idx, 'DECLARED'] = max(0.0, predicted_value)

# ستون‌های خروجی
output_cols = [
    'HOUR', 'DATE_MILADI', 'DATE_SHAMSI', 'POWER', 'CODE', 'UNIT_NO',
    'DAMA', 'ROTOOBAT', '12209_G13', '12210_G13', 'ebraz', 'importance_factor',
    'year', 'month', 'dayofweek', 'DECLARED'
]

# ذخیره خروجی
test[output_cols].to_csv('output_holtwinters_fixed.csv', index=False)

print("فایل خروجی output_holtwinters_fixed.csv با موفقیت ایجاد شد.")
print(f"تعداد ردیف‌های پیش‌بینی‌شده: {len(test)}")

هشدار: datetimeهای تکراری وجود دارد. در حال aggregation با میانگین POWER...
مقدار صفر/منفی در داده آموزشی برای 2024-01-01 18:00:00 — استفاده از Additive


C:\Users\pishva_r\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\holtwinters\model.py:85: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


مقدار صفر/منفی در داده آموزشی برای 2024-01-01 22:00:00 — استفاده از Additive


C:\Users\pishva_r\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\holtwinters\model.py:85: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


مقدار صفر/منفی در داده آموزشی برای 2024-01-02 00:00:00 — استفاده از Additive


...

C:\Users\pishva_r\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\holtwinters\model.py:85: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


مقدار صفر/منفی در داده آموزشی برای 2025-06-07 20:00:00 — استفاده از Additive


C:\Users\pishva_r\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\holtwinters\model.py:85: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


مقدار صفر/منفی در داده آموزشی برای 2025-06-07 22:00:00 — استفاده از Additive


C:\Users\pishva_r\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\holtwinters\model.py:85: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
